# Refinitiv Data Library for Python
## Content - Streaming Price Cache - Synchronous calls

This notebook demonstrates how to use the Streaming Price interface to retrieve streaming data from the Refinitiv Data Platform.

## Import the library and load credentials

Credentials used by this and the other tutorials notebooks are stored in the **Configuration/credentials.ipynb** file.     

You should have edited the **Configuration/credentials.ipynb** to set your credentials as part of the **Quick Start** step.

In [1]:
import refinitiv.data as rd
from pandas import DataFrame
from IPython.display import display, clear_output

%run ../../Configuration/credentials.ipynb

## Open the session of your choice

Use our helper function in the Credentials notebook, **open_session(session_type)**, to create and open a session to connect to the 
- Refinitiv Data Platform directly (session_type="rdp") or via 
- Eikon 4 or Refinitiv Workspace (session_type="desktop") or via a 
- local realtime infrastructure (session_type="deployed").

You can also set a default in the **credentials** notebook

In [2]:
open_session('rdp')

## Define and Open a cache for Streaming data
Define a Pricing object from the Streaming data feed - note the **get_stream()** call

In [3]:
# Define our Streaming Price object
streaming = rd.content.pricing.Definition(
    ['EUR=', 'GBP=', 'JPY='],
    fields=['BID', 'ASK']
).get_stream()
# Open the Stream - Once opened, the library starts caching the updates
streaming.open()
# When you want the current price, get a snapshot
streaming.get_snapshot()

,Instrument,BID,ASK
0,EUR=,1.1901,1.1905
1,GBP=,1.3904,1.3908
2,JPY=,110.47,110.5


#### Call get_snapshot() again, prices may change (assuming instruments are being traded/quoted)

In [4]:
streaming.get_snapshot()

,Instrument,BID,ASK
0,EUR=,1.1901,1.1905
1,GBP=,1.3904,1.3908
2,JPY=,110.47,110.5


### Alternative ways of accessing instruments + values

#### Direct Access to fields

In [5]:
streaming['EUR=']['BID']

1.1901

In [6]:
gbp = streaming['GBP=']
gbp['ASK']

1.3908

#### Iterate on fields

In [7]:
print('JPY=')
for field_name, field_value in streaming['JPY=']:
    print(f"\t{field_name} : {field_value}")

JPY=
	BID : 110.47
	ASK : 110.5


#### Iterate on Streaming instruments and fields

In [8]:
for instrument in streaming:
    print(instrument.name)
    for field_name, field_value in instrument:
        print(f"\t{field_name} : {field_value}")

EUR=
	BID : 1.1901
	ASK : 1.1905
GBP=
	BID : 1.3904
	ASK : 1.3908
JPY=
	BID : 110.47
	ASK : 110.5


### Close the Streaming Items when no longer required

In [9]:
streaming.close()

<StreamState.Closed: 1>

### Invalid or un-licensed instruments
What happens if you request using an invalid RIC or an instrument you are not entitled to?
Let's request a mixture of valid and invalid RICs

In [10]:
# Define our Streaming Price object
mixed = rd.content.pricing.Definition(
    ['EUR=', 'GBP=', 'DUMMY'],
    fields=['BID', 'ASK']
).get_stream()
# Open the Stream - Once opened, the library starts caching the updates
state = mixed.open()
mixed.get_snapshot()

,Instrument,BID,ASK
0,EUR=,1.1901,1.1905
1,GBP=,1.3904,1.3908
2,DUMMY,<NA>,<NA>


You can check the Status of any instrument, so lets check the invalid one

In [11]:
mixed['DUMMY'].state

<StreamState.Open: 3>

For an invalid instrument you should see:   
{'status': <StreamState.Closed: 1>, **'code': 'NotFound'**, 'message': '** The Record could not be found'}   
However, if you are not licensed for the instrument you would see something like:  
{'status': <StreamState.Closed: 1>, **'code': 'NotEntitled'**, 'message': 'A21: DACS User Profile denied access to vendor'}   
**NOTE**: Do not rely on the message text to remain unchanged, use the **code** value for any programmatic decision making.

## Close the session

In [12]:
close_session()